In [1]:
from p2pfl.node import Node
from p2pfl.learning.pytorch.mnist_examples.mnistfederated_dm import MnistFederatedDM
from p2pfl.learning.pytorch.mnist_examples.models.mlp import MLP
from collections import OrderedDict
import torch
import time


def test_convergence(x):
    n,r = x

    nodes = []
    
    for i in range(n):
        node = Node(MLP(),MnistFederatedDM())
        node.start()
        nodes.append(node)

    # Node Connection
    for i in range(len(nodes)-1):
        nodes[i+1].connect_to(nodes[i].host,nodes[i].port)
        time.sleep(0.1)

    # Check if they are connected
    for node in nodes:
        assert len(node.neightboors) == n-1

    # Start Learning
    nodes[0].set_start_learning(rounds=r,epochs=1)

    # Wait 4 results
    while True:
        time.sleep(1)
        finish = True
        for f in [node.round is None for node in nodes]:
            finish = finish and f

        if finish:
            break


    # Validamos Modelos obtenidos sean iguales
    model = None
    first = True
    for node in nodes:
        if first:
            model = node.learner.get_parameters()
            first = False
        else:
            for layer in model:
                a = torch.round(model[layer], decimals=2)
                b = torch.round(node.learner.get_parameters()[layer], decimals=2)
                assert torch.eq(a, b).all()

    # Cerrar
    for node in nodes:
        node.stop()
        time.sleep(.1) #Esperar por la asincronía
    
    return nodes
        
nodes = test_convergence((2,2))

Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:34757
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:53829
INFO:root:('127.0.0.1', 53829) Connected to 127.0.0.1:34757
INFO:root:('127.0.0.1', 34757) Conexión aceptada con 127.0.0.1:53829
DEBUG:root:Too mucho errors. ('127.0.0.1', 34757)
DEBUG:root:Last error: b'NUM_SAMPLES 54000 10000 -6952033739070900083\n'
DEBUG:root:Closed connection: ('127.0.0.1', 34757)


AssertionError: 

In [ ]:
from p2pfl.learning.pytorch.mnist_examples.mnistfederated_dm import MnistFederatedDM
from p2pfl.learning.pytorch.mnist_examples.models.cnn import CNN
from p2pfl.learning.pytorch.mnist_examples.models.mlp import MLP
from p2pfl.node import Node
import pytest
import time
import threading

nodes = []
        
def test_node_down_on_learning(n):

    # Node Creation
    nodes = []
    for i in range(n):
        node = Node(MLP(),MnistFederatedDM())
        node.start()
        nodes.append(node)

    # Node Connection
    for i in range(len(nodes)-1):
        nodes[i+1].connect_to(nodes[i].host,nodes[i].port)
        time.sleep(1)

    # Check if they are connected
    for node in nodes:
        assert len(node.neightboors) == n-1

    # Start Learning
    nodes[0].set_start_learning(rounds=2,epochs=0)

    # Stopping node
    time.sleep(0.3)
    nodes[1].stop()
    # Wait 4 results
    while True:
        time.sleep(1)
        finish = True
        for f in [node.round is None for node in nodes]:
            finish = finish and f

        if finish:
            break

    for node in nodes:
        node.stop()


nodes = []
for _ in range(6):
    test_node_down_on_learning(4)
    break


Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 56968
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 56969
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 56970
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 56971
INFO:root:(('127.0.0.1', 56968)) Conexión aceptada con ('127.0.0.1', 56969)
INFO:root:(('127.0.0.1', 56969)) Conexión aceptada con ('127.0.0.1', 56970)
INFO:root:(('127.0.0.1', 56970)) Conexión aceptada con ('127.0.0.1', 56968)
INFO:root:(('127.0.0.1', 56970)) Conexión aceptada con ('127.0.0.1', 56971)
INFO:root:(('127.0.0.1', 56971)) Conexión aceptada con ('127.0.0.1', 56969)
INFO:root:(('127.0.0.1', 56971)) Conexión aceptada con ('127.0.0.1', 56968)
INFO:root:(('127.0.0.1', 56968)) Broadcasting start learning...
INFO:root:(('127.0.0.1', 56968)) Sending Initial Model Weights
INFO:root:(('127.0.0.1', 56968)) Broadcasting model to 3 clients. (size: 578304 bytes)
INFO:root:

In [1]:
a = {('127.0.0.1', 35159): 185, ('127.0.0.1', 54181): 62}
a.items()

dict_items([(('127.0.0.1', 35159), 185), (('127.0.0.1', 54181), 62)])

In [1]:
from p2pfl.node import Node
from p2pfl.learning.pytorch.mnist_examples.mnistfederated_dm import MnistFederatedDM
from p2pfl.learning.pytorch.mnist_examples.models.mlp import MLP
from collections import OrderedDict
import torch
import time

def test_connect_to_train_running(four_nodes):
    n1,n2,n3,n4 = four_nodes

    # Connect Nodes (unless the n4)
    n1.connect_to(n2.host,n2.port)
    n3.connect_to(n1.host,n1.port)
    time.sleep(0.1) #Esperar por la asincronía

    n1.set_start_learning(2,1)

    time.sleep(2)
    
    n4.connect_to(n1.host,n1.port,full=False)

    time.sleep(2)

    #assert n4.round is not None

    # If the new node doesent know that learning is running, it will cause inanition in the other nodes (that are waiting for him to finish)

    while all([n1.round is not None for n in [n1,n2,n3,n4]]):
        time.sleep(0.1)

    for n in [n1,n2,n3,n4]:
        n.stop()
        time.sleep(0.1)
        
        
nodes = []
    
for i in range(4):
    node = Node(MLP(),MnistFederatedDM())
    node.start()
    nodes.append(node)
    
test_connect_to_train_running(nodes)


Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:38537
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:42513
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:41845
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:60271
INFO:root:('127.0.0.1', 38537) Connected to 127.0.0.1:42513
INFO:root:('127.0.0.1', 42513) Conexión aceptada con 127.0.0.1:38537
INFO:root:('127.0.0.1', 41845) Connected to 127.0.0.1:38537
INFO:root:('127.0.0.1', 38537) Conexión aceptada con 127.0.0.1:41845
INFO:root:('127.0.0.1', 42513) Connected to 127.0.0.1:41845
INFO:root:('127.0.0.1', 41845) Conexión aceptada con 127.0.0.1:42513
INFO:root:(('127.0.0.1', 38537)) Broadcasting start learning...
INFO:root:(('127.0.0.1', 38537)) Sending Initial Model Weights
INFO:root:(('127.0.0.1', 38537)) Broadcasting model to 2 clients. (size: 548864 bytes)
INFO:root:(('127.0.0.1', 42513)) Broadcasting Number of Samples...
INFO:root:(('127.0.0.1',

Testing: 0it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

INFO:root:('127.0.0.1', 38537) Conexión aceptada con 127.0.0.1:60271
INFO:root:('127.0.0.1', 60271) Connected to 127.0.0.1:38537
TO IMPLEMET WHEN THE TOPOLOGY WAS NOT FULLY CONNECTED
DEBUG:root:Closed connection: ('127.0.0.1', 60271)
DEBUG:root:Closed connection: ('127.0.0.1', 38537)
INFO:root:(('127.0.0.1', 38537)) Evaluated. Losss: 2.301456928253174, Metric: 0.11810000240802765. (Check tensorboard for more info)
INFO:root:(('127.0.0.1', 38537)) Training...
INFO:root:(('127.0.0.1', 42513)) Evaluated. Losss: 2.301456928253174, Metric: 0.11810000240802765. (Check tensorboard for more info)
INFO:root:(('127.0.0.1', 42513)) Training...


...

INFO:root:(127.0.0.1:38537) Model added (1/3) from ('127.0.0.1', 38537)
INFO:root:(127.0.0.1:42513) Model added (1/3) from ('127.0.0.1', 42513)
INFO:root:(('127.0.0.1', 38537)) Broadcasting model to 2 clients. (size: 548864 bytes)
INFO:root:(('127.0.0.1', 42513)) Broadcasting model to 2 clients. (size: 548864 bytes)
INFO:root:(127.0.0.1:41845) Model added (1/3) from ('127.0.0.1', 42513)
INFO:root:(127.0.0.1:38537) Model added (2/3) from ('127.0.0.1', 42513)
INFO:root:(127.0.0.1:41845) Model added (2/3) from ('127.0.0.1', 38537)
INFO:root:(127.0.0.1:42513) Model added (2/3) from ('127.0.0.1', 38537)
INFO:root:(127.0.0.1:41845) Model added (3/3) from ('127.0.0.1', 41845)
INFO:root:(127.0.0.1:41845) Agregating models.
INFO:root:(('127.0.0.1', 41845)) Broadcasting model to 2 clients. (size: 548864 bytes)
INFO:root:(('127.0.0.1', 41845)) Waiting other nodes.
INFO:root:(127.0.0.1:38537) Model added (3/3) from ('127.0.0.1', 41845)
INFO:root:(127.0.0.1:42513) Model added (3/3) from ('127.0.0.1

Testing: 0it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

INFO:root:(('127.0.0.1', 42513)) Evaluated. Losss: 0.07695037126541138, Metric: 0.9758999943733215. (Check tensorboard for more info)
INFO:root:(('127.0.0.1', 42513)) Finish!!.
INFO:root:(('127.0.0.1', 41845)) Evaluated. Losss: 0.07695037126541138, Metric: 0.9758999943733215. (Check tensorboard for more info)
INFO:root:(('127.0.0.1', 38537)) Evaluated. Losss: 0.07695037126541138, Metric: 0.9758999943733215. (Check tensorboard for more info)
INFO:root:(('127.0.0.1', 41845)) Finish!!.
INFO:root:(('127.0.0.1', 38537)) Finish!!.
INFO:root:Bajando el nodo, dejando de escuchar en 127.0.0.1 38537 y desconectándose de 2 nodos
DEBUG:root:Closed connection: ('127.0.0.1', 38537)
DEBUG:root:Closed connection: ('127.0.0.1', 38537)
DEBUG:root:Closed connection: ('127.0.0.1', 42513)
DEBUG:root:Closed connection: ('127.0.0.1', 41845)
INFO:root:Bajando el nodo, dejando de escuchar en 127.0.0.1 42513 y desconectándose de 1 nodos
DEBUG:root:Closed connection: ('127.0.0.1', 42513)
DEBUG:root:Closed connec

In [4]:
hash("msg" + str(datetime.now()))

3116227613539867172

In [3]:
from datetime import datetime

now = datetime.now()
str(now)

'2022-06-27 17:13:54.992782'

In [6]:
hash("caca".encode("utf-8"))

5682035749826620771

In [8]:
str(time.time())

'1656343107.3914704'

In [2]:
"-1".isdigit()

False